In [143]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [144]:
CSV_PATH = '../options_SPY_calls.csv'
df = pd.read_csv(CSV_PATH)
required_cols = {'S0','K','T','iv'}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f'Missing columns in CSV: {missing}')
S0 = float(df['S0'].iloc[0])
df['T'] = (np.floor(df['T'] * 10) / 10).astype(float)
df['iv'] = df['iv'].round(2)
df = df.drop_duplicates(subset=['K','T']).reset_index(drop=True)
print(f'Loaded {CSV_PATH} with {len(df)} rows | S0 = {S0:.4f}')


Loaded ../options_SPY_calls.csv with 1511 rows | S0 = 672.6500


In [145]:
K_grid = np.arange(S0 - 100.0, S0 + 100.0, 10)
T_grid = np.arange(0.0, 2.0, 0.1)
KK, TT = np.meshgrid(K_grid, T_grid)
print(f'Grid sizes -> K: {len(K_grid)}, T: {len(T_grid)}')


Grid sizes -> K: 20, T: 20


In [146]:
k_idx = np.abs(df['K'].to_numpy()[:, None] - K_grid).argmin(axis=1)
t_idx = np.abs(df['T'].to_numpy()[:, None] - T_grid).argmin(axis=1)
df['K_bin'] = K_grid[k_idx]
df['T_bin'] = T_grid[t_idx]
print (f'Binned data into {len(df["K_bin"].unique())} K bins and {len(df["T_bin"].unique())} T bins')

grouped = df.groupby(['T_bin','K_bin'], as_index=False)['iv'].mean()
IV_surface = np.full((len(T_grid), len(K_grid)), np.nan)
k_pos = {val: idx for idx, val in enumerate(K_grid)}
t_pos = {val: idx for idx, val in enumerate(T_grid)}
for row in grouped.itertuples(index=False):
    IV_surface[t_pos[row.T_bin], k_pos[row.K_bin]] = row.iv
print(f'Filled {grouped.shape[0]} grid cells (K step 10, T step 0.1)')

import numpy.ma as ma
mask = np.isnan(IV_surface)
for i in range(IV_surface.shape[0]):
    for j in range(IV_surface.shape[1]):
        if mask[i, j]:
            left = IV_surface[i, j - 1] if j - 1 >= 0 else np.nan
            right = IV_surface[i, j + 1] if j + 1 < IV_surface.shape[1] else np.nan
            neighbors = [val for val in (left, right) if not np.isnan(val)]
            if neighbors:
                IV_surface[i, j] = np.mean(neighbors)
mask = np.isnan(IV_surface)
IV_surface = np.where(mask, np.nanmean(IV_surface), IV_surface)


Binned data into 20 K bins and 13 T bins
Filled 260 grid cells (K step 10, T step 0.1)


In [147]:
fig = go.Figure(data=[go.Surface(x=KK, y=TT, z=np.nan_to_num(IV_surface, nan=np.nanmean(IV_surface)), colorscale='Viridis', opacity=0.8)])
fig.update_layout(
    title='Market Implied Volatility Surface',
    scene=dict(xaxis_title='Strike Price', yaxis_title='Time to Maturity (Years)', zaxis_title='Implied Volatility'),
    width=1000, height=600, template='plotly_dark'
)
fig.show()
